In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)


In [3]:
i = pd.read_csv('general')

In [5]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_manifacture(soup):
    
    try:
        # Outer Tag Object
        man = soup.find("span", attrs={"class":'a-size-base po-break-word'})

            # Inner NavigatableString Object
        man_value = man.text

            # Title as a string value
        man_string = man_value.strip()
    except:
        man_string = ""

    return man_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={"class":'a-price-whole'}).text
    except:
        price = ""
    return price



# get discount
def get_discount(soup):
    try:
        discount = soup.find("span", attrs={"class":'a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'}).text
    except:
        discount = ''
    return discount




# Function to extract Availability Status
def get_availability(soup):
    try:    
        available = soup.find("span", attrs={"class":'a-size-medium a-color-success'}).text
    except :
        available = "Not Available"

    return available
# Function to extract Product Rating


def get_rating(soup):

    try:
        rating = soup.find("span", attrs={"class":'a-icon-alt'}).text
    except:
        rating = "no rating"

    return rating
# Function to extract Product description
def get_desc(soup):

    try:
        desc = soup.find("div", attrs={"class":'a-section a-spacing-small a-spacing-top-small'}).text
    except:
        desc = "no description"
    return desc


def get_nrate(soup):
    try:
        
        nrate = soup.find("span", attrs={"id":"acrCustomerReviewText","class":'a-size-base'}).text
        
    except:
        
        nrate = "0 rates"

    return nrate




In [6]:
def data_generate(URL):
    # add your user agent 
    HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5' ,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Connection': 'keep-alive',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'}
    # The webpage URL
#     URL = 'https://www.amazon.eg/deal/912cee26?pf_rd_r=JTWKC8K5YSK39E4EDRJE&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=ee55a2a2-1309-4ad7-bf63-ee00919ab1d3&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_tlt_0_912cee26_dt_sl8_d3&pageNum=1&sortOption=price-asc-rank&language=en_AE'
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-size-base a-color-base a-link-normal a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[],"link":[], "manifacture":[],"price":[],"discount":[], "availability":[] ,"rating":[],"number_of_rating":[] , "description":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.eg" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['link'].append("https://www.amazon.eg" + link)
        d['manifacture'].append(get_manifacture(new_soup))
        d['price'].append(get_price(new_soup))
        d['discount'].append(get_discount(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['number_of_rating'].append(get_nrate(new_soup))
        d['description'].append(get_desc(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    return amazon_df

In [14]:
xiaomi = data_generate('https://www.amazon.eg/deal/fdf2c821?pf_rd_r=75AWJWVCM2YG0YZGNG70&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=a8e1ce50-f7b0-4c6b-bf71-28c9d0e5975a&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_img_7_fdf2c821_dt_sl8_5a&language=en_AE')
xiaomi

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description
0,"POCO X5 Pro 5G (Black 8GB RAM, 256 Storage) - 120Hz POLED Display | Snapdragon 778G processor | 67W turbo charging | 108MP pro-grade camera, Bluetooth, Wi-Fi, USB, NFC",https://www.amazon.eg/-/en/POCO-Pro-Black-8GB-Storage/dp/B0BRQMC66M?ref_=Oct_DLandingS_D_fdf2c821_0,Xiaomi,"14,333.",,In Stock,4.7 out of 5 stars,"6,837 ratings","Brand Name Xiaomi Wireless Provider Unlocked Operating System Android 12.0 Cellular Technology 5G Memory Storage Capacity 256 GB Connectivity Technology Bluetooth, Wi-Fi, USB, NFC Color grey Screen Size 6.67 Inches Wireless Network Technology LTE RAM Memory Installed 8 GB"
1,"Xiaomi POCO X5 Pro 5G (Yellow 8GB RAM, 256 Storage) - 120Hz POLED Display | Snapdragon 778G processor | 67W turbo charging | 108MP pro-grade camera",https://www.amazon.eg/-/en/Xiaomi-POCO-Pro-Yellow-Storage/dp/B0BRQKLQMH?ref_=Oct_DLandingS_D_fdf2c821_1,Xiaomi,"14,333.",,In Stock,4.7 out of 5 stars,"6,837 ratings","Brand Name Xiaomi Model Name Poco X5 Wireless Provider Unlocked Operating System Android 11.0 Cellular Technology 5G Memory Storage Capacity 256 GB Connectivity Technology Bluetooth, Wi-Fi, USB Color white Screen Size 6.67 Inches Wireless Network Technology GSM"
2,"Xiaomi Poco F5 PRO 5G 512GB + 12GB Global Version Black, USB",https://www.amazon.eg/-/en/Xiaomi-512GB-Global-Version-Black/dp/B0C3NKZT7P?ref_=Oct_DLandingS_D_fdf2c821_2,Xiaomi,"30,222.",,Not Available,4.6 out of 5 stars,163 ratings,Brand Name Xiaomi Wireless Provider Unlocked Operating System Android 4.1 Cellular Technology 5G Memory Storage Capacity 256 GB Connectivity Technology USB Color Black Screen Size 6.67 Inches Wireless Network Technology GSM SIM Card Slot Count Dual SIM


In [15]:
xiaomi['Description'] = "Phones"

In [21]:
oppo = data_generate('https://www.amazon.eg/deal/54c6084a?pf_rd_r=XCFMD8WY306T2DYK7AC6&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=a8e1ce50-f7b0-4c6b-bf71-28c9d0e5975a&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_img_168_54c6084a_dt_sl8_5a&language=en_AE')
oppo

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description
0,"realme 11 5G (Glory Gold, 8GB RAM, 256GB Storage) | Dynamic Ultra Smooth Display | Up to 8GB+8GB Dynamic RAM | 108MP 3× Zoom | 16MP Selfie Camera | Dimensity 6100+ 5G Processor | 67W SUPERVOOC Charge",https://www.amazon.eg/-/en/realme-Storage-Dimensity-Processor-SUPERVOOC/dp/B0CGF21RCY?ref_=Oct_DLandingS_D_54c6084a_0,realme,"9,999.",,In Stock,4.1 out of 5 stars,156 ratings,"Brand Name realme Model Name 11 5G Wireless Provider Unlocked for All Carriers Operating System realme UI 4.0 | Based on Android 13 Cellular Technology 5G, 4G, 4G VOLTE, 3G, 2G, GSM, WCDMA Memory Storage Capacity 256 GB Connectivity Technology 5G, 4G, 3G, EDGE, GPRS, Wi-Fi, Bluetooth Color Glory Gold Screen Size 6.72 Inches Wireless Network Technology LTE"
1,"Realme 9 Pro+ 5G (Sunrise Blue, 8GB RAM, 128GB Storage)",https://www.amazon.eg/-/en/Realme-Sunrise-Blue-128GB-Storage/dp/B09SVDPS2V?ref_=Oct_DLandingS_D_54c6084a_1,realme,"12,666.",,Not Available,4.0 out of 5 stars,386 ratings,Brand Name realme Model Name 9 Pro+ 5G Wireless Provider Unlocked for All Carriers Operating System Android 12.0 Cellular Technology 5G Memory Storage Capacity 128 GB Color Sunrise Blue Screen Size 6.4 Inches Wireless Network Technology Wi-Fi RAM Memory Installed 8 GB
2,Realme 9 Pro Dual-SIM 128GB ROM + 8GB RAM - 5G SmartPhone (Midnight Black),https://www.amazon.eg/-/en/Realme-Pro-Dual-SIM-128GB-ROM/dp/B09PBVW599?ref_=Oct_DLandingS_D_54c6084a_2,realme,"8,444.",,In Stock,4.1 out of 5 stars,127 ratings,"Brand Name realme Model Name Realme 9 Pro Wireless Provider Unlocked Operating System Android 10.0 Cellular Technology LTE Memory Storage Capacity 128 GB Connectivity Technology Bluetooth, Wi-Fi Color black Screen Size 6.6 Inches Wireless Network Technology GSM, LTE"


In [25]:
huawei = data_generate('https://www.amazon.eg/deal/89a471c9?pf_rd_r=BRJPFVRQVVKQ3WAZP271&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=a8e1ce50-f7b0-4c6b-bf71-28c9d0e5975a&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_img_191_89a471c9_dt_sl8_5a&language=en_AE')
huawei

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description
0,"HUAWEI nova Y61 Smartphone, Powerful Battery, Exquisite ID Design, Blue",https://www.amazon.eg/-/en/HUAWEI-Smartphone-Powerful-Battery-Exquisite/dp/B0BLW19BXL?ref_=Oct_DLandingS_D_89a471c9_0,HUAWEI,"5,555.",,In Stock,3.4 out of 5 stars,7 ratings,"Brand Name HUAWEI Model Name HUAWEI nova Y61 Wireless Provider Unlocked for All Carriers Operating System Android 12.0 Cellular Technology 4G Memory Storage Capacity 64 GB Connectivity Technology Bluetooth, Wi-Fi, USB Color light blue Screen Size 6.52 Inches Wireless Network Technology LTE"
1,جوال هواوي نوفا 10 SE - ذاكرة رام 8 جيجابايت - 108 ميجا بكسل، 256 جيجابايت - 4500 مللي امبير/الساعة - ستاري فضي,https://www.amazon.eg/Huawei-Nova-10-SE-4500MAh/dp/B0BHJG8X87?ref_=Oct_DLandingS_D_89a471c9_1,هواوي,"13,869.‎",-7‎%‎,متوفر,3.3 من 5 نجوم,14 تقييم,"اسم العلامة التجارية هواوي اسم الموديل نوفا 10 اس اي مزود الخدمة اللاسلكية جميع الناقلين نظام التشغيل MIUI 12 التكنولوجيا الخلوية 4G سعة تخزين الذاكرة 256 غيغابايت تكنولوجيا الاتصال Wi-Fi اللون ستاري سيلفر حجم الشاشة 6,67 بوصة تكنولوجيا الاتصالات اللاسلكية واي فاي, LTE"
2,"Huawei Nova 10 SE - 108MP - 8GB Ram, 256GB - 4500MAh - Starry Black",https://www.amazon.eg/-/en/Huawei-Nova-10-SE-4500MAh/dp/B0BHJHJN29?ref_=Oct_DLandingS_D_89a471c9_2,Huawei,"13,666.",-9%,In Stock,3.5 out of 5 stars,8 ratings,"Brand Name Huawei Model Name Nova 10 SE Wireless Provider Unlocked for All Carriers Operating System EMUI 12 Cellular Technology 4G Memory Storage Capacity 256 GB Connectivity Technology Wi-Fi Color Starry Black Screen Size 6.67 Inches Wireless Network Technology Wi-Fi, LTE"


In [29]:
vivo = data_generate('https://www.amazon.eg/deal/d7b43a19?pf_rd_r=1TFWW03W5ADH6VA2ZJ15&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=a8e1ce50-f7b0-4c6b-bf71-28c9d0e5975a&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_img_295_d7b43a19_dt_sl8_5a&language=en_AE')

In [30]:
vivo

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description
0,"Vivo Y02s Dual SIM, 3GB RAM, 32GB ROM, 4G LTE - Vibrant Blue",https://www.amazon.eg/-/en/Vivo-Y02s-Dual-SIM-32GB/dp/B0B9C3CZJ5?ref_=Oct_DLandingS_D_d7b43a19_0,Vivo,"4,149.",,In Stock,4.3 out of 5 stars,11 ratings,"Brand Name Vivo Model Name Y02s Wireless Provider Unlocked for All Carriers Operating System FunTouch OS 12, Android 12.0 Cellular Technology 4G Memory Storage Capacity 32 GB Connectivity Technology Bluetooth, Wi-Fi Color Blue Screen Size 6.51 Inches Wireless Network Technology LTE"


In [32]:
sam = data_generate('https://www.amazon.eg/deal/18b6d259?showVariations=true&pf_rd_r=1TFWW03W5ADH6VA2ZJ15&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=a8e1ce50-f7b0-4c6b-bf71-28c9d0e5975a&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_img_248_18b6d259_dt_sl8_5a&language=en_AE')
sam

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description
0,"Samsung Galaxy M34, Dual SIM, 8GB RAM, 128GB Storage, 5G, Blue - 1 year Warranty",https://www.amazon.eg/-/en/Samsung-Galaxy-Dual-128GB-Storage/dp/B0CKTTHVBN?ref_=Oct_DLandingS_D_18b6d259_0,SAMSUNG,"9,999.",-12%,In Stock,3.0 out of 5 stars,2 ratings,"Brand Name SAMSUNG Model Name Galaxy-M34 Wireless Provider Total Wireless Operating System Android 13.0 Cellular Technology 5G Memory Storage Capacity 128 GB Connectivity Technology Bluetooth, USB, NFC Color Blue Wireless Network Technology GSM, LTE RAM Memory Installed 8 GB"


In [38]:
realme = data_generate('https://www.amazon.eg/deal/19a83ef3?pf_rd_r=3X6NNQZ9DVMQB5DYVN51&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=a8e1ce50-f7b0-4c6b-bf71-28c9d0e5975a&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_tlt_321_19a83ef3_dt_sl8_5a&language=en_AE')

In [47]:
realme['Department'] = "Phones"

In [48]:
sam['Department'] = "Phones"

In [49]:
vivo['Department'] = "Phones"

In [50]:
huawei['Department'] = "Phones"

In [51]:
oppo['Department'] = "Phones"

In [ ]:
# xiaomi,oppo,huawei,vivo,sam,realme

In [52]:
general2 = pd.concat([i,xiaomi,oppo,huawei,vivo,sam,realme],ignore_index=True)

In [60]:
general2 = general2.drop(columns= ['Description'])

In [62]:
general2.to_csv('general2')

In [61]:
general2

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description,Department
0,"Lenovo TB-X306 LPDDR4X 2GB, 32GB (ZA6V0038EG)",https://www.amazon.eg/-/en/Lenovo-TB-X306-LPDDR4X-2GB-32GB-ZA6V0038EG/dp/B0BH4ZBC8P?ref_=Oct_DLandingS_D_912cee26_0,Lenovo,"4,460.",-16%,In Stock,5.0 out of 5 stars,1 rating,Brand Name Lenovo Model Name TB-X306 Screen Size 10.1 Inches Color Gray Hard Disk Size 32 GB RAM Memory Installed 2 GB Operating System Android Graphics Description Integrated Resolution 1280 x 800 Item Weight 420 Grams,Laptops
1,"itel 15.6 inch,FHD,Core-i3 1115G4,4GB+256GB SSD,50Wh, Dos",https://www.amazon.eg/-/en/itel-15-6-Core-i3-1115G4-256GB/dp/B0BSLP4X6T?ref_=Oct_DLandingS_D_912cee26_1,Itel,"10,510.",NaN,In Stock,Previous page,0 rates,Brand Name Itel Model Name itel 15 Screen Size 15.6 Inches Color Silver Hard Disk Size 256 GB CPU Model Core i3-9300T RAM Memory Installed 8 GB Operating System Windows 10 Graphics Description Integrated CPU Speed 3 GHz,Laptops
2,"Lenovo V15 Laptop - Intel Core i3-10110U, 8GB RAM, 1TB HDD, Integrated Intel UHD Graphics, 15.6"" FHD (1920x1080) TN 220nits Anti-glare, Dos - Iron Grey",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0C2GL2W8J?ref_=Oct_DLandingS_D_912cee26_2,LENOVO,"11,680.",-9%,In Stock,Previous page,0 rates,Brand Name LENOVO Model Name V15 - I3-10110U8G1TFHD Screen Size 15.6 Inches Color Grey Hard Disk Size 1 TB CPU Model Intel Core i3 RAM Memory Installed 8 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated,Laptops
3,"Lenovo V15 Laptop - Intel Core i3-10110U, 4GB RAM, 1TB HDD, Integrated Intel UHD Graphics, 15.6"" HD (1366x768) TN 220nits Anti-glare, Dos - Iron Grey",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0C2GF8C51?ref_=Oct_DLandingS_D_912cee26_3,LENOVO,"12,320.",NaN,In Stock,Previous page,0 rates,Brand Name LENOVO Model Name V15 - 82NB0015ED Screen Size 15.6 Inches Color Grey Hard Disk Size 1 TB CPU Model Intel Core i3 RAM Memory Installed 4 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated,Laptops
4,"Dell Vostro 3510 laptop - 11th Intel core i3-1115G4, 12 GB RAM, 256 GB SSD, Intel UHD Graphics, 15.6"" HD (1366X768) Anti-glare - Black",https://www.amazon.eg/-/en/Dell-Vostro-3510-laptop-Anti-glare/dp/B0BTGWCLXY?ref_=Oct_DLandingS_D_912cee26_4,DELL,"13,760.",-10%,In Stock,3.0 out of 5 stars,7 ratings,Brand Name DELL Model Name Vostro 3510 i3 12G 256 ub Screen Size 15.6 Inches Color Black Hard Disk Size 256 CPU Model Intel Core i3 RAM Memory Installed 12 GB Special Features Anti Glare Screen Graphics Description Integrated Graphics Coprocessor Intel UHD Graphics,Laptops
...,...,...,...,...,...,...,...,...,...,...
90,"Realme 9i dual sim mobile phone, *6.6 inch, 128gb rom, 4gb ram - prism black",https://www.amazon.eg/-/en/Realme-dual-mobile-phone-128gb/dp/B09R1VG156?ref_=Oct_DLandingS_D_19a83ef3_0,realme,"5,948.",-7%,In Stock,3.3 out of 5 stars,7 ratings,"Brand Name realme Model Name Realme 9i Wireless Provider T-Mobile Operating System Android 11.0 Cellular Technology 4G Memory Storage Capacity 128 GB Connectivity Technology Bluetooth, Wi-Fi, USB Color Prism Black Screen Size 6.6 Inches Wireless Network Technology GSM, Wi-Fi, CDMA",Phones
91,"Realme C21 Y, 4GB RAM, 64GB ROM, 4G LTE - Black",https://www.amazon.eg/-/en/Realme-C21-4GB-RAM-64GB/dp/B0BTTSCZBZ?ref_=Oct_DLandingS_D_19a83ef3_1,realme,"4,689.",-7%,In Stock,4.1 out of 5 stars,2 ratings,"Brand Name realme Model Name 6941399059252 Wireless Provider ALGAR Operating System Android 10.0 Cellular Technology 4G Memory Storage Capacity 64 GB Connectivity Technology Bluetooth, Wi-Fi, USB Color Black Screen Size 6.4 Inches Wireless Network Technology LTE",Phones
92,"Realme c30 dual sim, 4g lte, 32gb + 2gb *6.5 inches - lake blue",https://www.amazon.eg/-/en/Realme-c30-dual-32gb-inches/dp/B0BFBQLPZX?ref_=Oct_DLandingS_D_19a83ef3_2,realme,"3,569.",-7%,In Stock,5.0 out of 5 stars,2 rati

In [78]:
routers = data_generate('https://www.amazon.eg/deal/cb248ee3?pf_rd_r=9XV36TJFS16JJ3DWGCNV&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=a8e1ce50-f7b0-4c6b-bf71-28c9d0e5975a&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_img_15_cb248ee3_dt_sl8_5a&language=en_AE')
routers.shape

(13, 9)

In [71]:
# storage = data_generate('https://www.amazon.eg/deal/d1d19597?showVariations=true&pf_rd_r=9XV36TJFS16JJ3DWGCNV&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=a8e1ce50-f7b0-4c6b-bf71-28c9d0e5975a&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_img_19_d1d19597_dt_sl8_5a&language=en_AE')
# storage

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description
0,Lexar 256GB Micro SDXC 633x UHSI 95MB/s Memory Card with SD Adapter-,https://www.amazon.eg/-/en/Lexar-256GB-Micro-Memory-Adapter/dp/B07N5Z98YT?ref_=Oct_DLandingS_D_d1d19597_0,Lexar,665.,-7%,In Stock,4.4 out of 5 stars,9 ratings,"Brand Name Lexar Flash Memory Type Micro SDXC Memory Storage Capacity 256 GB Compatible Devices Gopro, Camera, Mobile, Mp4 Color black Special Features Write Protect Switch Item Weight 8 Grams Warranty Type 2 years Write Speed 95 MB per second Hardware Interface Secure Digital Card"
2,"Lexar Professional 1066x 256GB MicroSDXC Uhs-I Card with SD Adapter Silver Series, Up to 160MB/s Read 120MB/s Write",https://www.amazon.eg/-/en/Lexar-Professional-MicroSDXC-Adapter-Silver/dp/B08GGCMT2Z?ref_=Oct_DLandingS_D_d1d19597_2,Lexar,"1,011.",-16%,In Stock,4.8 out of 5 stars,975 ratings,Brand Name Lexar Flash Memory Type Micro SDXC Memory Storage Capacity 256 GB Color silver Special Features water proof Read Speed 160 Megabytes Per Second Item Weight 4 Grams Hardware Interface microSDXC Secure Digital Association Speed Class Class 10
3,"Lexar Professional 1066x 128GB Micro SDXC UHS-I Card w/SD Adapter (Silver Series, Up to 160MB/s Read 120MB/s Write)",https://www.amazon.eg/-/en/Lexar-Professional-1066x-Adapter-Silver/dp/B08GGCHKML?ref_=Oct_DLandingS_D_d1d19597_3,Lexar,548.,-16%,In Stock,4.7 out of 5 stars,650 ratings,Brand Name Lexar Model Name Professional Flash Memory Type Micro SDXC Memory Storage Capacity 128 GB Color black Read Speed 160 Item Weight 0.2 Ounces Hardware Interface microSDXC Secure Digital Association Speed Class Class 10
4,"Lexar Professional 1667x 128GB SDXC UHS-II Card, Up To 250MB/s Read",https://www.amazon.eg/-/en/Lexar-Professional-1667x-128GB-UHS-II/dp/B07SZ3GCC6?ref_=Oct_DLandingS_D_d1d19597_4,Lexar,"1,011.",-10%,In Stock,4.7 out of 5 stars,"5,666 ratings","Brand Name Lexar Model Name UHS-II Flash Memory Type SDXC Memory Storage Capacity 128 MB Compatible Devices Camera Color Black Special Features water_proof, x_ray_proof, temperature_proof Read Speed 250 Megabytes Per Second Item Weight 0.01 Kilograms Warranty Type Lifetime"
5,"Lexar JumpDrive V100 128GB USB 3.0 Flash Drive, Gray (LJDV100-128ABNL)",https://www.amazon.eg/-/en/Lexar-JumpDrive-128GB-Flash-LJDV100-128ABNL/dp/B08HQ8P3JB?ref_=Oct_DLandingS_D_d1d19597_5,Lexar,285.,-18%,In Stock,4.5 out of 5 stars,55 ratings,Brand Name Lexar Memory Storage Capacity 128 GB Hardware Interface USB 3.0 Special Features Lightweight Read Speed 150 Megabytes Per Second Connectivity Technology USB Color grey Flash Memory Type USB Flash Drive
6,Kingston DataTraveler microDuo 3C 128GB USB-C & USB-A Flash Drive | Speeds up to 200 MB/s | USB 3.2 Gen 1 | Duo Connector | DTDUO3CG3/128GB,https://www.amazon.eg/-/en/Kingston-DataTraveler-128GB-Connector-DTDUO3CG3/dp/B0B1W3VLQK?ref_=Oct_DLandingS_D_d1d19597_6,kingston,638.,-12%,In Stock,4.6 out of 5 stars,170 ratings,Brand Name kingston Memory Storage Capacity 128 GB Hardware Interface USB 3.2 Gen 1 Special Features Dual USB Connector Read Speed 200 Megabytes Per Second Connectivity Technology USB Color Purple + Silver Flash Memory Type Memory Stick Duo
7,"Lexar Jumpdrive Fingerprint F35 Usb 3.0 Flash Drive, 32Gb Capacity",https://www.amazon.eg/-/en/Lexar-Jumpdrive-Fingerprint-Flash-Capacity/dp/B082T4XGBQ?ref_=Oct_DLandingS_D_d1d19597_7,Lexar,835.,-16%,In Stock,3.9 out of 5 stars,10 ratings,"Brand Name Lexar Memory Storage Capacity 32 GB Hardware Interface microSDHC, USB 3.0 Special Features Lightweight Read Speed 150 Megabytes Per Second Connectivity Technology USB Color Black Model Name JumpDrive Flash Memory Type USB"
8,"Keendex kx2336 hard drive with cable, 3tb - transparent",https://www.amazon.eg/-/en/Keendex-kx2336-hard-drive-cable/dp/B0979TC3CH?ref_=Oct_DLandingS_D_d1d19597_8,Keendex,232.,-10%,In Stock,3.8 out of 5 stars,13 ratings,Brand Name Keendex Color Transparent Material Mixed Materials Item Weight 100 Grams Co

In [74]:
storage.shape

(22, 9)

In [79]:
# storage,routers


In [81]:
storage['Department'] = 'Storage'

In [82]:
routers['Department'] = 'Networking'

In [83]:
general3 = pd.concat([i,storage,routers],ignore_index=True)
general3

,Unnamed: 0,title,link,manifacture,price,discount,availability,rating,number_of_rating,description,Department
0,0.0,"Lenovo TB-X306 LPDDR4X 2GB, 32GB (ZA6V0038EG)",https://www.amazon.eg/-/en/Lenovo-TB-X306-LPDDR4X-2GB-32GB-ZA6V0038EG/dp/B0BH4ZBC8P?ref_=Oct_DLandingS_D_912cee26_0,Lenovo,"4,460.",-16%,In Stock,5.0 out of 5 stars,1 rating,Brand Name Lenovo Model Name TB-X306 Screen Size 10.1 Inches Color Gray Hard Disk Size 32 GB RAM Memory Installed 2 GB Operating System Android Graphics Description Integrated Resolution 1280 x 800 Item Weight 420 Grams,Laptops
1,1.0,"itel 15.6 inch,FHD,Core-i3 1115G4,4GB+256GB SSD,50Wh, Dos",https://www.amazon.eg/-/en/itel-15-6-Core-i3-1115G4-256GB/dp/B0BSLP4X6T?ref_=Oct_DLandingS_D_912cee26_1,Itel,"10,510.",NaN,In Stock,Previous page,0 rates,Brand Name Itel Model Name itel 15 Screen Size 15.6 Inches Color Silver Hard Disk Size 256 GB CPU Model Core i3-9300T RAM Memory Installed 8 GB Operating System Windows 10 Graphics Description Integrated CPU Speed 3 GHz,Laptops
2,2.0,"Lenovo V15 Laptop - Intel Core i3-10110U, 8GB RAM, 1TB HDD, Integrated Intel UHD Graphics, 15.6"" FHD (1920x1080) TN 220nits Anti-glare, Dos - Iron Grey",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0C2GL2W8J?ref_=Oct_DLandingS_D_912cee26_2,LENOVO,"11,680.",-9%,In Stock,Previous page,0 rates,Brand Name LENOVO Model Name V15 - I3-10110U8G1TFHD Screen Size 15.6 Inches Color Grey Hard Disk Size 1 TB CPU Model Intel Core i3 RAM Memory Installed 8 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated,Laptops
3,3.0,"Lenovo V15 Laptop - Intel Core i3-10110U, 4GB RAM, 1TB HDD, Integrated Intel UHD Graphics, 15.6"" HD (1366x768) TN 220nits Anti-glare, Dos - Iron Grey",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0C2GF8C51?ref_=Oct_DLandingS_D_912cee26_3,LENOVO,"12,320.",NaN,In Stock,Previous page,0 rates,Brand Name LENOVO Model Name V15 - 82NB0015ED Screen Size 15.6 Inches Color Grey Hard Disk Size 1 TB CPU Model Intel Core i3 RAM Memory Installed 4 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated,Laptops
4,4.0,"Dell Vostro 3510 laptop - 11th Intel core i3-1115G4, 12 GB RAM, 256 GB SSD, Intel UHD Graphics, 15.6"" HD (1366X768) Anti-glare - Black",https://www.amazon.eg/-/en/Dell-Vostro-3510-laptop-Anti-glare/dp/B0BTGWCLXY?ref_=Oct_DLandingS_D_912cee26_4,DELL,"13,760.",-10%,In Stock,3.0 out of 5 stars,7 ratings,Brand Name DELL Model Name Vostro 3510 i3 12G 256 ub Screen Size 15.6 Inches Color Black Hard Disk Size 256 CPU Model Intel Core i3 RAM Memory Installed 12 GB Special Features Anti Glare Screen Graphics Description Integrated Graphics Coprocessor Intel UHD Graphics,Laptops
...,...,...,...,...,...,...,...,...,...,...,...
109,NaN,"MERCUSYS Halo H30G(3-Pack) AC1300 Whole Home Mesh Wi-Fi System 1.3 Gbps Dual Band WiFi Gigabit Wireless Router 1300 Mbps Speed Coverage, White",https://www.amazon.eg/-/en/MERCUSYS-H30G-Gigabit-Wireless-Coverage/dp/B09H6ZLDXW?ref_=Oct_DLandingS_D_cb248ee3_13,MERCUSYS,"4,753.",-5%,In Stock,4.3 out of 5 stars,107 ratings,"Brand Name MERCUSYS Model Name AC1300 Special Features Access Point Mode, MESH Frequency Band Class Dual-Band Wireless Communication Standard 802.11n, 802.11b, 802.11a, 802.11ac, 802.11g Compatible Devices Security Camera, Personal Computer, Smartphone Frequency 5 GHz Recommended Uses For Product Gaming, Home Included Components 1× Quick Installation Guide, 1× RJ45 Ethernet Cable, 3× Halo H30G Units, 3× Power Adapters Connectivity Technology Wi-Fi, Ethernet",Networking
110,NaN,USB 2.0 Ethernet 10/100Mbps RJ45 Network Lan Adapter Card,https://www.amazon.eg/-/en/Ethernet-100Mbps-RJ45-Network-Adapter/dp/B08YNPN53P?ref_=Oct_DLandingS_D_cb248ee3_14,Generic,89.,,In Stock,2.7 out of 5 stars,33 ratings,"Brand Name Generic Hardware Interface USB, Ethernet, USB 2.0 Color White Compatible Devices Laptop, Desktop Data Transfer Rate 100 Megabits Per Second Item Weight 0.06 K

In [84]:
general3.to_csv('general3')